<H1>Synapses</H1>
<P>Simulation of a chemical synapse between principal cells</P> 

In [ ]:
%pylab inline

In [ ]:
from neuron import h
h.load_file('stdrun.hoc') # using neuron library used by the iv graphics

<H3>Model specification </H3>

<P> 1. Topology and biophysical properties of a single-compartment</P>

In [ ]:
class Cell:
    def __init__(self, myname):
        """
        Create a single compartiment with Hodgkin-Huxley 
        conductances.
        """
        self.soma = h.Section(name = myname)
        self.soma.diam = 3.57 # PI*R^2 area 10 um^2
        
        self.soma.insert('pas') # RC basic circuit
        self.soma.e_pas = -65 # resting membrane potential

        self.soma.insert('hh') # HH conductances
        

In [ ]:
#create two cells
neuron_pre = Cell(myname = 'pre')
neuron_post = Cell(myname = 'post')

<H3>Instrumentation</H3>

<P> 2. Stimulate the presynaptic cell</P>

In [ ]:
stim = h.IClamp(0.5,neuron_pre.soma) 
stim.amp = 1 # in nA
stim.delay = 5 # in ms
stim.dur = 0.5 # in ms

<P> 3. Create postsynaptic side of the synapse </P>

In [ ]:
postsyn = h.ExpSyn(neuron_post.soma(0.5))
postsyn.tau = .1 # decay time constant
postsyn.e = 0 # reversal potential, by default is set to 0


<P> 4. Create the presynaptic side (transmission delay, and synaptic weight) <P>

In [ ]:
syn= h.NetCon(neuron_pre.soma(0.5)._ref_v, postsyn, sec = neuron_pre.soma)
syn.weight[0] = 5

In [ ]:
syn.delay = 2.5 # in ms (default 1)

<H2>Simulation control</H3>

<p> 5. Record, run and plot

In [ ]:
t,v1,v2 = h.Vector(), h.Vector(), h.Vector()
t.record(h._ref_t)
v1.record(neuron_pre.soma(0.5)._ref_v)
v2.record(neuron_post.soma(0.5)._ref_v)
h.finitialize(-65)
h.continuerun(20)

In [ ]:
plt.plot(t,v1,t,v2)
plt.xlabel('Time (ms)'), plt.ylabel('Voltage (mV)')
plt.show()

In [ ]:
# delete Hogdkin-Huxley conductances in the post-synaptic neuron
neuron_post.soma.gnabar_hh = 0.0
neuron_post.soma.gkbar_hh = 0.0

In [ ]:
def myrun(tstop):
    """
    Prepare runs the simulation until tstop
    and plot voltages from neuron_pre and
    neuron_post
    """
    # create hoc vectors
    t,v1,v2 = h.Vector(), h.Vector(), h.Vector()
    
    t.record(h._ref_t) # simulation time
    
    v1.record( neuron_pre.soma(0.5)._ref_v ) # voltage neuron_pre
    v2.record( neuron_post.soma(0.5)._ref_v ) # voltage neuron_post
    h.finitialize(-65)
    h.continuerun(tstop)
    
    # plot voltage vs time for both neurons
    plt.plot(t, v1,label='pre', color = 'darkblue')
    plt.plot(t, v2, label='post', color = 'orange')
    plt.legend()
    plt.xlabel('Time (ms)'), plt.ylabel('Voltage (mV)')
    plt.show()
    

In [ ]:
myrun(tstop = 20)